In [1]:
import pandas as pd
import datetime
from datetime import datetime
from dateutil.relativedelta import relativedelta
import locale
import os
import os.path
from os import path

from unidecode import unidecode
import matplotlib.pyplot as plt

locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')

'pt_BR.utf8'

In [2]:
def format_date(date, series = False, interval = 'm'):
    
    if series:
        date = date.dt
    
    if interval == 'm':
        format_date = (date.strftime("%b")+date.strftime("%y"))
    elif interval == 'w':
        format_date = (date.strftime("%d")+'-'+date.strftime("%b")+date.strftime("%y"))
        
    return format_date

In [3]:
def aggrupe_datasets(v_start_date, v_end_date, period, from_folder, to_folder, models):

    for model in models:

        df_results = []
        new_v_end_date = v_start_date
        new_v_start_date = v_start_date

        i = 0

        while new_v_end_date < v_end_date:

            new_v_end_date = (datetime.strptime(new_v_start_date, "%Y-%m-%d") + 
                      relativedelta(months=period-1)).strftime('%Y-%m-%d')

            t_end_date = (datetime.strptime(new_v_start_date, "%Y-%m-%d") - 
                          relativedelta(months=1)).strftime('%Y-%m-%d')

            t_start_date = (datetime.strptime(t_end_date, "%Y-%m-%d") - 
                            relativedelta(months=period-1)).strftime('%Y-%m-%d')

            train_folder = (f"T_{t_start_date}_{t_end_date}")
            validation_folder = (f"V_{new_v_start_date}_{new_v_end_date}")


            file_path = f'../{from_folder}/{train_folder}/{validation_folder}/{model}.csv'

            if path.exists(file_path):
                if i == 0:
                    df_results = pd.read_csv(file_path)
                    new_start_t_folder = t_start_date
                    new_start_v_folder = new_v_start_date
                    i += 1
                else:
                    df_results = pd.concat([df_results, pd.read_csv(file_path)])
            else:
                print(f"Não há informações sobre treino {train_folder} e validação {validation_folder} para {model}")

            new_v_start_date = (datetime.strptime(new_v_end_date, "%Y-%m-%d") + 
                                relativedelta(months=1)).strftime('%Y-%m-%d')
            
        new_end_t_folder = t_end_date
        new_end_v_folder = new_v_end_date

        new_path = f'../{to_folder}/T_{new_start_t_folder}_{new_end_t_folder}/V_{new_start_v_folder}_{new_end_v_folder}'

        if len(df_results)>0:
            if not os.path.exists(new_path):
                os.makedirs(new_path)

            df_results.to_csv(f"{new_path}/{model}.csv")

In [4]:
met_comp = ["MEQ", "MENS", "MVE"]
prog_lin = ["MAD", "MADD", "MINMAX", "DMINMAX"]
todos = met_comp + prog_lin

v_start_date="2021-01-01"
v_end_date="2021-12-01"
period = 3

aggrupe_datasets(v_start_date, v_end_date, period, "T3m_V3m_tri", "T3m_V3m_anual", todos)